In [1]:
import os
import time
import uuid
import cv2
import shutil
import albumentations as alb

import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model

In [40]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

def load_labels(label_path):
    '''function used to load labels 
    base on input which is path to file'''
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    return [label['class']]

In [81]:
### loading pretrained CNN model - VGG16

include_top=False configuration exclude the fully connected layers, leaving only the convolutional base of the VGG16 model

In [82]:
vgg = VGG16(include_top=False)
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

# Loading images

In [42]:
train_images = tf.data.Dataset.list_files('aug_data4\\train\\photos\\*.jpg', shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))  # Resize the images to a shape of (120, 120)
train_images = train_images.map(lambda x: x/255)                          # Normalize images by dividing each pixel value by 255

In [43]:
test_images = tf.data.Dataset.list_files('aug_data4\\test\\photos\\*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))    # Resize the images to a shape of (120, 120)
test_images = test_images.map(lambda x: x/255)                            # Normalize images by dividing each pixel value by 255

In [44]:
val_images = tf.data.Dataset.list_files('aug_data4\\val\\photos\\*.jpg', shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))     # Resize the images to a shape of (120, 120)
val_images = val_images.map(lambda x: x/255)                             # Normalize images by dividing each pixel value by 255

In [45]:
train_images.as_numpy_iterator().next()

array([[[0.30751637, 0.30751637, 0.29967323],
        [0.4362745 , 0.4490196 , 0.45784312],
        [0.5035948 , 0.53496736, 0.5428105 ],
        ...,
        [0.62385625, 0.6552288 , 0.6669935 ],
        [0.6245098 , 0.65588236, 0.66764706],
        [0.6313726 , 0.67058825, 0.6784314 ]],

       [[0.33529413, 0.33333334, 0.3254902 ],
        [0.44607842, 0.46372548, 0.45392156],
        [0.5281046 , 0.54771245, 0.55947715],
        ...,
        [0.62941176, 0.6607843 , 0.672549  ],
        [0.6362745 , 0.66764706, 0.67941177],
        [0.6326799 , 0.667974  , 0.67777795]],

       [[0.3156863 , 0.31241828, 0.30457515],
        [0.4392157 , 0.4627451 , 0.45882353],
        [0.52156866, 0.5529412 , 0.56078434],
        ...,
        [0.6333333 , 0.66862744, 0.6745098 ],
        [0.6362745 , 0.66764706, 0.6754902 ],
        [0.63529414, 0.6666667 , 0.6745098 ]],

       ...,

       [[0.47026145, 0.52516335, 0.55457515],
        [0.46862745, 0.527451  , 0.55490196],
        [0.47843137, 0

# Loading labels

In [46]:
train_labels = tf.data.Dataset.list_files('aug_data4\\train\\labels\\*.json', shuffle=False)  # Create a dataset containing file paths to the JSON label files
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8]))       # Use the `map` function to apply a load_labels function

In [47]:
test_labels = tf.data.Dataset.list_files('aug_data4\\test\\labels\\*.json', shuffle=False) # Create a dataset containing file paths to the JSON label files
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8]))      # Use the `map` function to apply a load_labels function

In [48]:
val_labels = tf.data.Dataset.list_files('aug_data4\\val\\labels\\*.json', shuffle=False) # Create a dataset containing file paths to the JSON label files
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8]))      # Use the `map` function to apply a load_labels function

# test, train and validation data sets

In [50]:
train = tf.data.Dataset.zip((train_images, train_labels))  # Combine the train_images and train_labels
train = train.shuffle(5000)                                # Shuffle the elements of the train dataset randomly
train = train.batch(8)                                     # Group the elements of the train dataset into batches of size 8
train = train.prefetch(4)

In [51]:
test = tf.data.Dataset.zip((test_images, test_labels))     # Combine the test_images and test_labels
test = test.shuffle(1300)                                  # Shuffle the elements of the test dataset randomly
test = test.batch(8)                                       # Group the elements of the test dataset into batches of size 8
test = test.prefetch(4)

In [52]:
val = tf.data.Dataset.zip((val_images, val_labels))        # Combine the val_images and val_labels
val = val.shuffle(1000)                                    # Shuffle the elements of the val dataset randomly
val = val.batch(8)                                         # Group the elements of the val dataset into batches of size 8
val = val.prefetch(4)

# Building model

In [54]:
def build_model(): 
    input_layer = Input(shape=(120,120,3))                        # Define the input layer with a shape of (120, 120, 3)
                                                                  # This specifies the expected shape of the input images
    
    vgg = VGG16(include_top=False)(input_layer)                   # Create the convolutional base of the model using VGG16

    # Classification Model  
    f1 = GlobalMaxPooling2D()(vgg)                                # Apply global max pooling to the output of the VGG16 model
    class1 = Dense(2048, activation='relu')(f1)                   # Add a dense layer with 2048 units and ReLU activation
    class2 = Dense(1, activation='sigmoid')(class1)               # Add a dense layer with 1 unit and sigmoid activation
    
    
    fist_tracker = Model(inputs = input_layer, outputs=[class2])  # Create a model that takes input from the input_layer and outputs the class2 predictions

    return fist_tracker

In [55]:
fist_tracker = build_model()         # Create the `fist_tracker` model by calling the `build_model` function

In [56]:
fist_tracker.summary()               # Print a summary of the `fist_tracker` model's architecture

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 120, 120, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_max_pooling2d_1 (Glo  (None, 512)              0         
 balMaxPooling2D)                                                
                                                                 
 dense_2 (Dense)             (None, 2048)              1050624   
                                                                 
 dense_3 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 15,767,361
Trainable params: 15,767,361
Non-trainable params: 0
_______________________________________________

In [63]:
batches_per_epoch = len(train)            # Calculate the number of batches per epoch
initial_learning_rate = 0.0001            # Set the initial learning rate to 0.0001
lr_decay = (1./0.75 -1)                   # Calculate the learning rate decay factor

0.33333333333333326

In [64]:
# Create a learning rate schedule using ExponentialDecay
# This schedule will decrease the learning rate over time in an exponential manner
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(   
    initial_learning_rate,
    decay_steps = batches_per_epoch,
    decay_rate=lr_decay,
    staircase=True)

# Create an Adam optimizer with the specified learning rate schedule
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [65]:
classloss = tf.keras.losses.BinaryCrossentropy()    # Create a binary cross-entropy loss function

In [66]:
class FistTracker(Model): 
    def __init__(self, fisttracker,  **kwargs):  # The fisttracker argument represents the pre-built model passed to the class
        super().__init__(**kwargs)
        self.model = fisttracker                 # The model is assigned to the 'self.model' attribute
        

    def compile(self, opt, classloss, **kwargs):
        super().compile(**kwargs)                # Configure the model for training
        self.closs = classloss                   # The classloss argument represents the loss function used for training
        self.opt = opt                           # The opt argument represents the optimizer used for training
        
    
    def train_step(self, batch, **kwargs): 
        X, y = batch                             # Retrieve the input features 'X' and target labels 'y' from the batch
        
        with tf.GradientTape() as tape: 
            classes = self.model(X, training=True)   # Set training=True to enable training-specific behavior e.g., applying dropout
            total_loss = self.closs(y[0], classes)   # Calculate the total loss using the provided loss function 'self.closs'
            grad = tape.gradient(total_loss, self.model.trainable_variables)  # Compute the gradients of the total loss

        opt.apply_gradients(zip(grad, self.model.trainable_variables))        # Update the model's trainable variables
        return {"total_loss":total_loss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes = self.model(X, training=False)  # Forward pass: obtain the predicted classes using the model
        total_loss = self.closs(y[0], classes)   # Calculate the total loss using the provided loss function 'self.closs'

        return {"total_loss":total_loss}
        
    def call(self, X, **kwargs):                 # This method forwards the input 'X' through the model and returns the output
        return self.model(X, **kwargs)

In [67]:
model = FistTracker(fist_tracker)     # The fist_tracker model is passed as an argument to initialize the FistTracker class

In [68]:
model.compile(opt, classloss)         # Compile the model for training

In [69]:
logdir='logs'                         # Define the directory path where the TensorBoard logs will be saved

In [70]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) # The 'log_dir' parameter specifies the directory path where the logs will be saved

In [71]:
hist = model.fit(train, epochs = 4, validation_data = val, callbacks=[tensorboard_callback])

Epoch 1/4
105/105 [==============================] - 274s 3s/step - total_loss: 0.5992 - val_total_loss: 0.8469
Epoch 2/4
105/105 [==============================] - 277s 3s/step - total_loss: 0.3991 - val_total_loss: 0.7421
Epoch 3/4
105/105 [==============================] - 282s 3s/step - total_loss: 0.1614 - val_total_loss: 6.8578e-05
Epoch 4/4
105/105 [==============================] - 286s 3s/step - total_loss: 0.0685 - val_total_loss: 0.1509


In [72]:
fist_tracker.save('fist_tracker_aug4_1.h5')

In [77]:
facetracker = load_model('fist_tracker_aug4_1.h5')

In [86]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10532), started 0:01:12 ago. (Use '!kill 10532' to kill it.)